In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

In [8]:
df = pd.read_csv(r"C:\Users\LAPTA\AIoT-2025\TP1\smoke_detection_iot.csv")
df.head()

,Unnamed: 0,UTC,Temperature[C],Humidity[%],TVOC[ppb],eCO2[ppm],Raw H2,Raw Ethanol,Pressure[hPa],PM1.0,PM2.5,NC0.5,NC1.0,NC2.5,CNT,Fire Alarm
0,0,1654733331,20.000,57.36,0,400,12306,18520,939.735,0.0,0.0,0.0,0.0,0.0,0,0
1,1,1654733332,20.015,56.67,0,400,12345,18651,939.744,0.0,0.0,0.0,0.0,0.0,1,0
2,2,1654733333,20.029,55.96,0,400,12374,18764,939.738,0.0,0.0,0.0,0.0,0.0,2,0
3,3,1654733334,20.044,55.28,0,400,12390,18849,939.736,0.0,0.0,0.0,0.0,0.0,3,0
4,4,1654733335,20.059,54.69,0,400,12403,18921,939.744,0.0,0.0,0.0,0.0,0.0,4,0


In [9]:
print(df.info())

print(df.isnull().sum())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62630 entries, 0 to 62629
Data columns (total 16 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Unnamed: 0      62630 non-null  int64  
 1   UTC             62630 non-null  int64  
 2   Temperature[C]  62630 non-null  float64
 3   Humidity[%]     62630 non-null  float64
 4   TVOC[ppb]       62630 non-null  int64  
 5   eCO2[ppm]       62630 non-null  int64  
 6   Raw H2          62630 non-null  int64  
 7   Raw Ethanol     62630 non-null  int64  
 8   Pressure[hPa]   62630 non-null  float64
 9   PM1.0           62630 non-null  float64
 10  PM2.5           62630 non-null  float64
 11  NC0.5           62630 non-null  float64
 12  NC1.0           62630 non-null  float64
 13  NC2.5           62630 non-null  float64
 14  CNT             62630 non-null  int64  
 15  Fire Alarm      62630 non-null  int64  
dtypes: float64(8), int64(8)
memory usage: 7.6 MB
None
Unnamed: 0        0
UTC   

In [10]:
features = [
    "Temperature[C]", "Humidity[%]", "TVOC[ppb]", "eCO2[ppm]",
    "Raw H2", "Raw Ethanol", "Pressure[hPa]",
    "PM1.0", "PM2.5", "NC0.5", "NC1.0", "NC2.5", "CNT"
]
target = "Fire Alarm"

X = df[features]
y = df[target]


In [11]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

In [12]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

mean_values = scaler.mean_
std_values = scaler.scale_

print("Mean (μ):", mean_values)
print("Std (σ):", std_values)


Mean (μ): [1.59531151e+01 4.85743019e+01 1.92288434e+03 6.67743913e+02
 1.29423010e+04 1.97542505e+04 9.38631948e+02 1.00887285e+02
 1.86073921e+02 4.90209271e+02 2.05457793e+02 8.13197535e+01
 1.05209288e+04]
Std (σ): [1.43580385e+01 8.82118774e+00 7.75589505e+03 1.90378221e+03
 2.71168609e+02 6.06975952e+02 1.32451443e+00 9.25303530e+02
 1.99152789e+03 4.25941567e+03 2.23248476e+03 1.09529003e+03
 7.60459566e+03]


In [13]:
clf = LogisticRegression(max_iter=1000, class_weight='balanced')  
clf.fit(X_train_scaled, y_train)

y_pred = clf.predict(X_test_scaled)


print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))


Accuracy: 0.983793709085103
              precision    recall  f1-score   support

           0       0.95      0.99      0.97      3575
           1       1.00      0.98      0.99      8951

    accuracy                           0.98     12526
   macro avg       0.98      0.99      0.98     12526
weighted avg       0.98      0.98      0.98     12526



In [14]:
weights = clf.coef_[0]  # مصفوفة الأوزان W
bias = clf.intercept_[0]  # الانترسبت b

print("Weights (W):", weights)
print("Bias (b):", bias)

Weights (W): [ -0.46196775   1.02379758 -29.88140562   3.4092624    0.3868544
  -4.51676325  -1.96708206   1.242099     0.4984672    1.97559229
   0.44472092  -0.15287263  16.61662482]
Bias (b): 12.010646163992215


In [15]:
print("const float MEAN[N_FEATURES] = {", ', '.join(map(str, mean_values)), "};")
print("const float STD[N_FEATURES]  = {", ', '.join(map(str, std_values)), "};")
print("const float WEIGHTS[N_FEATURES] = {", ', '.join(map(str, weights)), "};")
print("const float BIAS = ", bias, ";")

const float MEAN[N_FEATURES] = { 15.953115080632283, 48.57430185214753, 1922.8843405716111, 667.7439126616638, 12942.300973974134, 19754.250459045186, 938.6319484871467, 100.88728524668689, 186.0739208446432, 490.20927071690875, 205.45779255149284, 81.31975353265209, 10520.92878812071 };
const float STD[N_FEATURES]  = { 14.358038477662703, 8.82118774162391, 7755.895054463778, 1903.7822129584472, 271.16860881062263, 606.9759523422715, 1.3245144335765542, 925.3035296966241, 1991.52788902251, 4259.415669501792, 2232.484760401612, 1095.2900289597956, 7604.5956649702775 };
const float WEIGHTS[N_FEATURES] = { -0.46196774642136845, 1.0237975815826177, -29.88140561609288, 3.4092623995956473, 0.38685440295360274, -4.516763249773937, -1.967082064475801, 1.2420990016465594, 0.498467200020788, 1.97559228730834, 0.4447209207791269, -0.15287262705417967, 16.616624817319465 };
const float BIAS =  12.010646163992215 ;
